In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports and environment variables
Set user_root to point to the root of your work directory which is not on the GoogleCloud instance

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import os.path as path
import re
import os
import sys
import shutil
import pathlib
from tqdm.notebook import tqdm_notebook

user_root = '/content/drive/MyDrive/Msc/'
work_dir = path.join(user_root, 'Final project/Pancancer survival analysis of cancer hallmark genes')

data_dir = path.join(work_dir, 'data')
Expression_tables_dir = path.join(data_dir, "Expression_tables")
results_dir = path.join(work_dir, 'results')
local_data_folder = "/content/data"
local_expression_folder = path.join(local_data_folder, "expression_tables")
local_results_folder = "/content/results"

data_for_lorsi_dir = path.join(work_dir, 'data_for_LoRSI')

query_size = 2000


In [ ]:
# Key = site name as in the data set from GDC
# Value = cancer name as in the article
# cancer_dict = {'Bladder':'Bladder',
#                'Breast':'Breast',
#                'Colon':'Colon',
#                'Esophagus':'Esophagus',
#                'Pancreas':'Pancreas',
#                'Stomach':'Stomach',
#                'Brain':'Glioblastoma',
#                'Cervix uteri':'Cervical',
#                'Connective, subcutaneous and other soft tissues':'Sarcoma',
#                'Liver and intrahepatic bile ducts':'Liver',
#                'Ovary':'Ovarium',
#                'Prostate gland':'Prostate',
#                'Skin':'Melanoma',
#                'Thyroid gland':'Thyroid',
#                'Uterus, NOS':'Uterine',
#                'Heart, mediastinum, and pleura':'Thymoma',
#                'Hematopoietic and reticuloendothelial systems':'AML',
#                'Larynx':'Head and Neck'
# }

In [ ]:
# Constract site_list from the data that was already downloaded and processed
site_list = []
total_cases = 0
for site in os.listdir(data_dir):
  try:
    meta_data_file = list(pathlib.Path(path.join(data_dir, site)).glob("gdc_sample_sheet.tsv"))[0]
  except Exception as e:
    print(e)
    print(site)
  else:
    if path.exists(meta_data_file):
      temp_df = pd.read_csv(meta_data_file, sep="\t")
      num_cases = temp_df.shape[0]
      if num_cases >= min_num_cases:
        site_list.append(site.split('_')[0])
        total_cases += num_cases
        print(f"{site} has {num_cases} cases")
print(f"Total {total_cases} cases")
print(len(site_list))
site_list

Adrenal gland_data has 289 cases
Bladder_data has 405 cases
Brain_data has 783 cases
Breast_data has 1074 cases
Bronchus and lung_data has 1226 cases
Cervix uteri_data has 414 cases
Colon_data has 453 cases
Corpus uteri_data has 545 cases
Hematopoietic and reticuloendothelial systems_data has 2133 cases
Kidney_data has 1188 cases
Liver and intrahepatic bile ducts_data has 402 cases
Lymph nodes_data has 519 cases
Ovary_data has 377 cases
Pancreas_data has 324 cases
Prostate gland_data has 493 cases
Skin_data has 472 cases
Stomach_data has 372 cases
Thyroid gland_data has 503 cases
list index out of range
.ipynb_checkpoints
list index out of range
Expression_tables
Total 11972 cases
18


['Adrenal gland',
 'Bladder',
 'Brain',
 'Breast',
 'Bronchus and lung',
 'Cervix uteri',
 'Colon',
 'Corpus uteri',
 'Hematopoietic and reticuloendothelial systems',
 'Kidney',
 'Liver and intrahepatic bile ducts',
 'Lymph nodes',
 'Ovary',
 'Pancreas',
 'Prostate gland',
 'Skin',
 'Stomach',
 'Thyroid gland']

# mRNAseq_data_processing

##Process the data with R code 
Initial R code was downloaded from https://github.com/adam-nagy91/pancancer_survival_analysis

**R cannot work with GoogleDrive.** Need to copy from Drive to the local disc any file that need to be processed by R. Aftyer R is done, need to copy the output files from the local disc to Drive.  

**Do the initial processing of the "htseq.counts.gz" files in Python.** Merge all "htseq.counts.gz" files of a specific site into a DataFrame were in gene identifiers are used as the Index and each column represents a "htseq.counts.gz" file. The merged DataFrame is then saved in TSV format to be processed by the R code.

In [ ]:
def merge_htseq_files(site):
  files = list(pathlib.Path(path.join(data_dir, f"{site}_data")).glob("*.htseq.counts*"))
  table_df = pd.read_csv(files[0].as_posix(), sep="\t", header=None)
  table_df.columns = ['ENSEMBL', files[0].name]
  table_df.set_index('ENSEMBL', inplace=True)
  table_df.drop(columns=[table_df.columns[0]], inplace=True)
  for file_ in files:
    temp_df = pd.read_csv(file_.as_posix(), sep="\t", header=None)
    temp_df.columns = ['ENSEMBL', file_.name]
    temp_df.set_index('ENSEMBL', inplace=True)

    table_df = table_df.merge(temp_df, left_index=True, right_index=True)
  return table_df

def copy_folder(src, dst, filter="*.*"):
  if not path.exists(dst):
    os.mkdir(dst)
  
  for file_ in list(pathlib.Path(src).glob(filter)):
    shutil.copyfile(file_, path.join(dst, file_.name))
    


## Prepare the data for the R code
* Constract the local folders:
  * for the input data - `local_data_folder`, and a sub directory per tumor 
  * for the intermidiate resilts (R code has two stages) - `local_expression_folder`
  * for the final results of the R code - `local_results_folder`
* Create the merged HTSEQ files and save them under the local data folder. By default, if a merged HTSEQ file already exists in Drive, we will not create a new file and just copy the existing one to the local data folder. Set `create_new = True` to force re-generating a new merged HTSEQ file.
* Copy the meta-data file (gdc_sample_sheet.tsv) of each tumor to its folder

In [ ]:
create_new = False

if not path.exists(local_data_folder):
  os.mkdir(local_data_folder)

for tumor in site_list:
  print(tumor)
  tumor_dir = path.join(data_dir, f"{tumor}_data")
  local_tumor_folder = path.join(local_data_folder, f"{tumor}_data")
  if not path.exists(local_tumor_folder):
    os.mkdir(local_tumor_folder)
  merged_file = path.join(tumor_dir, f"{tumor}_merged_htseq.tsv")
  if create_new or not path.exists(merged_file):
    table_df = merge_htseq_files(tumor)
    table_df.to_csv(path.join(local_tumor_folder, f"{tumor}_merged_htseq.tsv"), sep="\t")
    shutil.copyfile(path.join(local_tumor_folder, f"{tumor}_merged_htseq.tsv"), merged_file)
  else:
    shutil.copyfile(merged_file, path.join(local_tumor_folder, f"{tumor}_merged_htseq.tsv"))
  
  shutil.copyfile(path.join(tumor_dir, "gdc_sample_sheet.tsv"), path.join(local_tumor_folder, "gdc_sample_sheet.tsv"))

if not path.exists(local_expression_folder):
  os.mkdir(local_expression_folder)

if not path.exists(local_results_folder):
  os.mkdir(local_results_folder)

Adrenal gland
Bladder
Brain
Breast
Bronchus and lung
Cervix uteri
Colon
Corpus uteri
Hematopoietic and reticuloendothelial systems
Kidney
Liver and intrahepatic bile ducts
Lymph nodes
Ovary
Pancreas
Prostate gland
Skin
Stomach
Thyroid gland


In [ ]:
# Copy the merged tables back to GoogleDrive
for tumor in site_list:
  if create_new or not path.exists(path.join(data_dir, f"{tumor}_data", f"{tumor}_merged_htseq.tsv")):
    copy_folder(path.join(local_data_folder, f"{tumor}_data"), path.join(data_dir, f"{tumor}_data"), filter="*_merged_htseq.tsv")


## Load the required libraries for the R code

In [ ]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("AnnotationDbi")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("DESeq2")


R[write to console]: Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)

R[write to console]: 'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


R[write to console]: Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)

R[write to console]: 'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


R[write to console]: Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)

R[write to console]: 'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


R[write to console]: Bioconductor version 3.14 (BiocManager 1.30.16), R 4.1.3 (2022-03-10)



In [ ]:
%%R

# Required packages
library("AnnotationDbi")
library("org.Hs.eg.db")
library("DESeq2")

# ==== Merge separated htseq count files into one table ====
# Under data_folder it is expect ted to have per tumor folder named "<tumor>_data".
# Each folder should hold the htseq files and a tsv file with the meta data

data_folder = "/content/data"
Expression_tables_folder = paste(data_folder, "/", "expression_tables", sep="")
result_folder = "/content/results"
tumor_folders = list.files(path=data_folder, pattern = "_data", full.names = T)
tumor_type = list.files(path=data_folder, pattern = "_data")
tumor_type = strsplit(tumor_type, "_")
tumor_type = sapply(tumor_type, "[[", 1)
tumor_type

 [1] "Adrenal gland"                                
 [2] "Bladder"                                      
 [3] "Brain"                                        
 [4] "Breast"                                       
 [5] "Bronchus and lung"                            
 [6] "Cervix uteri"                                 
 [7] "Colon"                                        
 [8] "Corpus uteri"                                 
 [9] "Hematopoietic and reticuloendothelial systems"
[10] "Kidney"                                       
[11] "Liver and intrahepatic bile ducts"            
[12] "Lymph nodes"                                  
[13] "Ovary"                                        
[14] "Pancreas"                                     
[15] "Prostate gland"                               
[16] "Skin"                                         
[17] "Stomach"                                      
[18] "Thyroid gland"                                


## First stage of R - Pre process the per tumor merged HTSEQ file
The pre processing has two stages:  

>**Stage1**:

* Create a Dataframe per cancer site where the index is the gene name and the columns are the gene count per HTSEQ file name
* Map the index names (genes) from "ENSEMBL" to "SYMBOL" using the "org.Hs.eg.db" library. Since the mapping is many to one, remove all duplicate SYMBOls by keeping the first occurance
* Remove any row (gene) that is missing a value in any of the columns (cases)
* Use the Meta-data file to map the column names from HTSEQ file name to CASE_ID
* Normalizing htseq counts data using DESeq

>**Stage2**:  

* Scale each count by dividing it by the mean of its column (case) and multiply by 1000  

In [ ]:
%%R
# Loop over all tumors
for (t in 1:length(tumor_type)){
  print(tumor_type[t])
  tumor_folder = paste(data_folder, "/", tumor_type[t], "_data",sep="")
  print(tumor_folder)
  files = list.files(path = tumor_folder, pattern = "_merged_htseq.tsv", full.names = T)
  table = read.table(files[1], sep="\t", header=T, row.names=1, check.names=F)
  ens_id = as.character(rownames(table))
  ens_id = strsplit(ens_id, ".", fixed = T)
  ens_id = sapply(ens_id, "[[", 1)
  rownames(table) = ens_id
  table$"V1" = mapIds(org.Hs.eg.db, keys=as.character(rownames(table)), column="SYMBOL", keytype="ENSEMBL", multiVals="first")
  table_temp = table
  table$"V1"=NULL
  table_annot = cbind(table_temp$"V1", table)
  # Remove non-full rows
  table_annot = table_annot[complete.cases(table_annot), ]
  # Remove duplicated genes, keep the first occurence
  table_annot = table_annot[!duplicated(as.character(table_annot[[1]])), ] # The first transcript variant was kept!
  # Set gene names to be the index
  rownames(table_annot) = as.character(table_annot[[1]])
  # Remove the first column
  table_annot[[1]] = NULL
  # ==== Remove miRNAs from the table ==== 
  table_annot = table_annot[!grepl("^MIR", rownames(table_annot)), ]
  # ==== Processing patient's ids ====
  meta_files = list.files(path = tumor_folder, pattern = "gdc_sample_sheet.tsv", full.names = T, recursive = T)
  meta = read.table(meta_files[1], header = T, sep = "\t", check.names = F)
  # tomach = c("01A", "01B", "01C")
  # meta = meta[grep(as.character(meta$`Sample ID`), pattern = paste(tomach, collapse = "|")),]
  rownames(meta) = meta$`file_name`
  #meta_list = rownames(meta)
  # Replace column names from file name to case ID
  for (b in 1:ncol(table_annot)) {
      colnames(table_annot)[[b]] = as.character(meta[colnames(table_annot)[[b]], "Case ID"])
  }
  # ==== Removing duplicated samples ==== 
  duplicated_samples = colnames(table_annot[, duplicated(colnames(table_annot))])
  table_annot = table_annot[, !duplicated(colnames(table_annot))] # The first duplicant was kept!!!!!
  # ==== Normalizing htseq counts data using DESeq ====
  counttable = table_annot
  counttable1 = estimateSizeFactorsForMatrix(counttable)
  normalized_counts = t(t(counttable)/counttable1)
  normalized_counts = round(normalized_counts, digits=0)
  normalized_counts = as.data.frame(normalized_counts)
  # ==== Export data ====
  write.table(normalized_counts, paste(paste(Expression_tables_folder, "/", tumor_type[t], sep=""), "mRNA_expression.txt", sep = "_"), sep= "\t", col.names = NA, quote = F)
}


[1] "Adrenal gland"
[1] "/content/data/Adrenal gland_data"


R[write to console]: 'select()' returned 1:many mapping between keys and columns



[1] "Bladder"
[1] "/content/data/Bladder_data"


R[write to console]: 'select()' returned 1:many mapping between keys and columns



[1] "Brain"
[1] "/content/data/Brain_data"


...

R[write to console]: 'select()' returned 1:many mapping between keys and columns



[1] "Stomach"
[1] "/content/data/Stomach_data"


R[write to console]: 'select()' returned 1:many mapping between keys and columns



[1] "Thyroid gland"
[1] "/content/data/Thyroid gland_data"


R[write to console]: 'select()' returned 1:many mapping between keys and columns



In [ ]:
# Copy the Expression_tables back to GoogleDriv for backup and future use
copy_folder(local_expression_folder, Expression_tables_dir)


In [ ]:
# If starting the run from this point, copy the Expression_tables from GoogleDriv
for site in site_list:
  tumor_expression_files = list(pathlib.Path(Expression_tables_dir).glob(f"{site}_*"))
  for file_ in tumor_expression_files:
    shutil.copyfile(file_, path.join(local_expression_folder, file_.name))


In [ ]:
%%R

#==== Scaling ====
setwd(Expression_tables_folder)
files = list.files(pattern = "mRNA_expression", recursive = T)
tumor_type = strsplit(files, "_")
tumor_type = sapply(tumor_type, "[[", 1)
mrna = read.table(files[1], header = T, sep = "\t", check.names = F, stringsAsFactors = F, row.names = 1)
genes = as.character(rownames(mrna))
for (c in 1:length(files)){
  mrna_c = read.table(files[c], header = T, sep = "\t", check.names = F, stringsAsFactors = F, row.names = 1)
  colmeans_c = colMeans(mrna_c)
  mrna_c = as.data.frame(mapply(function(x,y) round(x/y*1000, digits = 0), x = mrna_c, y = colmeans_c))
  rownames(mrna_c) = genes
  write.table(mrna_c, paste(tumor_type[c], "mRNA_scaled_expression.txt", sep = "_"), sep= "\t", col.names = NA, quote = F)
}

In [ ]:
# Copy the scaled Expression_tables back to GoogleDriv for backup and future use
copy_folder(local_expression_folder, Expression_tables_dir, filter="*mRNA_scaled_expression.txt")


## Second stage of R - univariate_survival_analysis
For each of the pre-selected genes, find the best cutoff threshold  on each tumor.
Flow to find the best cutoff per gene per tumor:
* Create a list of 50 potencial cutoff values, from the 25% quantile to the 75% quantile in steps of 1
* For each cutoff value, tag the cases into two groups and calculate the P-Value
* Select the cutoff with the best P-Value

**Outputs:**
* A classification file per tumor that classifies each case per gene according to the selected cutoff of that gene  
  * Row per case  
  * Column per gene 
* Overall survival 
* Cutoff tables per tumor per gene. May be used for FDR processing.



In [ ]:
%%R

install.packages('beeswarm')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/beeswarm_0.4.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 24395 bytes (23 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write

In [ ]:
%%R

# Required PACKAGES
library(survival)
library(beeswarm)
library(ggplot2)

# ==== SURVIVAL FUNCTION ====
bestcutoff <- function(datavector, clintable){# , gene, tumor_type) {
  #print(tumor_type)
  #print(gene)
  # list of 50 values from the 25th quantile to the 75th quantile
  breaks <- quantile(datavector, probs = seq(0.25, 0.75, by= 0.01))
  # Call cutoff for each value in break
  cutoff.table <- t(sapply(breaks, function(z) cutoff(datavector = datavector, cutpoint = z, clintable = clintable)))
  colnames(cutoff.table) <- c("cutoff", "pvalue")
  #write.table(cutoff.table, paste(result_folder, "/", paste(tumor_type, gene, "cutoff_table.txt", sep = "_"),sep=""), sep="\t")
  #cutoff.table
  #cutoff.table[order(cutoff.table[, 2]), "cutoff"][1]
  cutoff.table
}

cutoff <- function(datavector, cutpoint, clintable) {
  term <- cut(x = datavector, breaks = c(min(datavector), cutpoint, max(datavector)), labels = F, include.lowest = T)
  # cox <- summary(coxph(Surv(surv_time, surv_events) ~ term, data = clintable))
  logrank <- survdiff(Surv(surv_time, surv_events) ~ term, data = clintable)
  # c(cutpoint, cox$sctest[3])
  c(cutpoint, 1 - pchisq(logrank$chisq, length(logrank$n) - 1))
}

In [ ]:

# Copy expression tables, clinical data, and gene table from GoogleDrive to local drive
if not path.exists(local_data_folder):
  os.mkdir(local_data_folder)
if not path.exists(local_expression_folder):
  os.mkdir(local_expression_folder)
for exp_file in os.listdir(Expression_tables_dir):
  if not path.exists(path.join(local_expression_folder, exp_file)):
    shutil.copyfile(path.join(Expression_tables_dir, exp_file), path.join(local_expression_folder, exp_file))
for tumor in site_list:
  tumor_dir = path.join(data_dir, f"{tumor}_data")
  local_tumor_folder = path.join(local_data_folder, f"{tumor}_data")
  if not path.exists(local_tumor_folder):
    os.mkdir(local_tumor_folder)
  #if not path.exists(path.join(local_tumor_folder, "gdc_sample_sheet.tsv")):
  shutil.copyfile(path.join(tumor_dir, "gdc_sample_sheet.tsv"), path.join(local_tumor_folder, "gdc_sample_sheet.tsv"))
gene_file = path.join(work_dir, 'Candidate_genes.txt')
shutil.copyfile(gene_file, path.join(local_data_folder, "Candidate_genes.txt"))

if not path.exists(local_results_folder):
  os.mkdir(local_results_folder)
for tumor in site_list:
  tumor_results_dir = path.join(results_dir, f"{tumor}_cutoffs")
  if not path.exists(tumor_results_dir):
    os.mkdir(tumor_results_dir)
  local_tumor_results_folder = path.join(local_results_folder, f"{tumor}_cutoffs")
  if not path.exists(local_tumor_results_folder):
    os.mkdir(local_tumor_results_folder)

In [ ]:
%%R

# List files

exp_files = list.files(path = Expression_tables_folder, pattern = "mRNA_scaled_expression.txt", full.names = T, recursive = T)
#clin_file = list.files(path = getwd(), pattern = "PanCancer_clinical_table_all190708.txt", full.names = T, recursive = T)
cancer_genes_file = list.files(path = data_folder, pattern = "Candidate_genes.txt", full.names = T, recursive = T)

# Reading cancer hallmark genes (707 unique genes)
cancer_genes = read.table(cancer_genes_file, sep = "\t",  header = F, check.names = F)

# Reading clinical table
#clinical = read.table(clin_file, sep = "\t", header = T, row.names = 1, check.names = F)

surv_analysis = as.data.frame(matrix(nrow = length(rownames(cancer_genes)), ncol = length(exp_files)*5+3))


In [ ]:
%%R

tumor_type_offset = 6
num_cols_per_gene = 5

for (a in 1:length(exp_files)){
  
  # Reading expression table
  expression = read.table(exp_files[a], header = T, sep = "\t", check.names = F, row.names = 1)
  
  tumor_type = strsplit(exp_files[a], split = "[/_]")
  tumor_type = sapply(tumor_type, "[[", tumor_type_offset)
  print(tumor_type)
  tumor_folder = paste(data_folder,"/",tumor_type,"_data",sep="")
  clin_file = paste(tumor_folder,"/","gdc_sample_sheet.tsv",sep="")
  #clin_file = list.files(path = tumor_folder, pattern = "gdc_sample_sheet.tsv", full.names = T, recursive = T)
  clinical = read.table(clin_file, sep = "\t", header = T, row.names = 1, check.names = F)
  
  cases = intersect(colnames(expression), rownames(clinical))
  expression = expression[, cases]
  clintable = clinical[cases, ]
  
  clintable$event = as.numeric(mapply(function(x) if (x=='Alive'){0} else {1},x = clintable$vital_status))
  
  surv_time = as.numeric(clintable[[8]]) # OS = column 3; RFS = column 5
  surv_events = as.numeric(clintable[[14]])#  OS = column 4; RFS = column 6
  
  new_table = clinical[cases,] # create classification table
  
  for (b in 1:nrow(cancer_genes)){
    tryCatch(
      expr = {
        # ==== Selecting genes ====
        selected_exp = as.numeric(expression[as.character(cancer_genes[b, 1]), ])
        
        # selected_exp = as.numeric(expression["BAX", ])
        
        # ==== Selecting the best cutoff to dividing patients into high and low expression groups: ==== 
        # cutoff.point = as.numeric(bestcutoff(datavector = selected_exp, clintable = clintable, gene=as.character(cancer_genes[b, 1], tumor_type)))
        cutoff.table = bestcutoff(datavector = selected_exp, clintable = clintable) #, gene=as.character(cancer_genes[b, 1], tumor_type))
        cutoff.point = cutoff.table[order(cutoff.table[, 2]), "cutoff"][1]
        write.table(cutoff.table, paste(result_folder, paste(tumor_type, "cutoffs", sep="_"), paste(tumor_type, gene, "cutoff_table.txt", sep = "_"),sep="/"), sep="\t")
        # ==== Dividing patients into low and high expression groups: ==== 
        exp_category = c()
        
        for (c in 1:length(selected_exp)){
          
          if (selected_exp[[c]] >= cutoff.point[1]){
            
            exp_category[[c]] = "High"
            
          } else {
            
            exp_category[[c]] = "Low"
            
          }
          
        }

        exp_category = factor(exp_category, levels = c("Low", "High"))

        # add results to classification table        
        gene = cancer_genes[b, 1]
        new_table[gene] = exp_category

        # ==== Cox-regression ==== 
        cox_result = coxph(Surv(surv_time, surv_events) ~ as.factor(exp_category))
        logrank_result = survdiff(Surv(surv_time, surv_events) ~ as.factor(exp_category))
        
        # Result tables of univariate analysis
        # p.val <- summary(cox_result)$sctest['pvalue']
        p.val <- 1 - pchisq(logrank_result$chisq, length(logrank_result$n) - 1)
        surv_analysis[b, a*num_cols_per_gene] = as.numeric(p.val)
        colnames(surv_analysis)[a*num_cols_per_gene] = paste(tumor_type, "pvalue", sep = "_")
        
        surv_analysis[b, a*num_cols_per_gene+1] = as.numeric(round(summary(cox_result)$conf.int[1], digits = 2))
        colnames(surv_analysis)[a*num_cols_per_gene+1] = paste(tumor_type, "HR", sep = "_")
        
        surv_analysis[b, a*num_cols_per_gene+2] = as.numeric(round(summary(cox_result)$conf.int[3], digits = 2))
        colnames(surv_analysis)[a*num_cols_per_gene+2] = paste(tumor_type, "CI_low", sep = "_")
        
        surv_analysis[b, a*num_cols_per_gene+3] = as.numeric(round(summary(cox_result)$conf.int[4], digits = 2))
        colnames(surv_analysis)[a*num_cols_per_gene+3] = paste(tumor_type, "CI_high", sep = "_")
        
        surv_analysis[b, a*num_cols_per_gene+4] = as.numeric(cutoff.point[1])
        colnames(surv_analysis)[a*num_cols_per_gene+4] = paste(tumor_type, "cutoff", sep = "_")
        
        
        # ==== Plot expression (Beeswarm plot)==== 
        exp_category_bee = cut(selected_exp, breaks = quantile(selected_exp, c(0, 0.25, 0.75, 1)), labels = c("low", "mid", "low"), include.lowest = T)
        
        ggplot(mapping = aes(x="BAX", y = selected_exp, colour = exp_category_bee)) +
          geom_jitter(width = 0.5, show.legend = F, size = 3) +
          scale_color_manual(values=c("#999999", "black")) +
          labs(y = "Expression", title = "BAX") +
          theme_bw() +
          theme(axis.title.x = element_blank(),
                axis.line = element_line(colour = "black"),
                plot.title = element_text(size = 20, hjust = 0.5))
        
      },
      
      error = function(e){
        e = toString(unlist(e))
        error_line = data.frame(b, e, Sys.time(), stringsAsFactors = F)
        # write.table(error_line, paste("/home/adam/Documents/TCGA_PanCancer/Error_Warnings/", paste(tumor_type, "error_pancancer.txt", sep = "_"), sep = ""), append = T, sep = "\t", quote = F, row.names = F, col.names = F)
      }
      
      # warning = function(w){
      # w = toString(unlist(w))
      # warning_line = data.frame(b, w, Sys.time(), stringsAsFactors = F)
      # # write.table(warning_line, paste("/home/adam/Documents/TCGA_PanCancer/Error_Warnings/", paste(tumor_type, "warning_pancancer.txt", sep = "_"), sep = ""), append = T, sep = "\t", quote = F, row.names = F, col.names = F)
      # }
    )
  }
  
  print(tumor_type) 
  write.table(new_table, paste(result_folder, "/", paste(tumor_type, "logrank_classification_results.txt", sep = "_"),sep=""), sep= "\t",col.names = T, quote = F)
}
surv_analysis[[1]] = as.character(cancer_genes[[1]])
surv_analysis[[2]] = as.character(cancer_genes[[1]])
surv_analysis[[3]] = as.character(cancer_genes[[1]])

#setwd("./results")
write.table(surv_analysis,paste(result_folder, "/logrank_OS_survival_results.txt", sep = ""), sep= "\t", col.names = NA, quote = F)


[1] "Adrenal gland"
[1] "Adrenal gland"
[1] "Bladder"
[1] "Bladder"
[1] "Brain"
[1] "Brain"
[1] "Breast"
[1] "Breast"
[1] "Bronchus and lung"
[1] "Bronchus and lung"
[1] "Cervix uteri"
[1] "Cervix uteri"
[1] "Colon"
[1] "Colon"
[1] "Corpus uteri"
[1] "Corpus uteri"
[1] "Hematopoietic and reticuloendothelial systems"
[1] "Hematopoietic and reticuloendothelial systems"
[1] "Kidney"
[1] "Kidney"
[1] "Liver and intrahepatic bile ducts"
[1] "Liver and intrahepatic bile ducts"
[1] "Lymph nodes"
[1] "Lymph nodes"
[1] "Ovary"
[1] "Ovary"
[1] "Pancreas"
[1] "Pancreas"
[1] "Prostate gland"
[1] "Prostate gland"
[1] "Skin"
[1] "Skin"
[1] "Stomach"
[1] "Stomach"
[1] "Thyroid gland"
[1] "Thyroid gland"


In [ ]:
# Copy the results to GoogleDrive
copy_folder(local_results_folder, results_dir, filter="*.txt")

for site in site_list:
  local_cutoffs_folder = path.join(local_results_folder, f"{site}_cutoffs")
  cutoffs_dir = path.join(results_dir, f"{site}_cutoffs")
  if path.exists(local_cutoffs_folder):
    copy_folder(local_cutoffs_folder, cutoffs_dir)

# Prepare the results for LoRSI
Per tumor per gene, create a CSV file with three columns, 'event', 'time', and 'group' 

* Create a folder according to `data_for_lorsi_dir`
* Create a sub folder per tumor
* per tumor:
  * Read the classification file that was created by the second stage of the R code
  * Map the 'vital_status' column to 'event'
  * Map the 'OS' column to 'time'
  * Per gene:
    * Map the 'class' column to 'group'
    * Save a CSV file per gene

In [ ]:
if not path.exists(data_for_lorsi_dir):
  os.mkdir(data_for_lorsi_dir)

In [ ]:
def map_vital_status_to_event(vital_status):
  if vital_status == 'Dead':
    return 1
  else:
    return 0

def map_os_to_time(os):
  if os > 0:
    return os
  else:
    return 240

def map_class_to_group(class_):
  if class_ == 'Low':
    return 'A'
  else:
    return 'B'

def prepare_tumor_data_for_lorsi(tumor, tumor_classification_df):
  print(f"Start preparing data for {tumor}")
  tumor_data_dir = path.join(data_for_lorsi_dir, tumor)
  if not path.exists(tumor_data_dir):
    os.mkdir(tumor_data_dir)

  df_for_lorsi = pd.DataFrame(columns=['event', 'time', 'group'])
  df_for_lorsi['event'] = tumor_classification_df['vital_status'].apply(map_vital_status_to_event)
  df_for_lorsi['time'] = tumor_classification_df['OS'].apply(map_os_to_time)
  gene_list = tumor_classification_df.columns[13:]
  for gene in gene_list:
    df_for_lorsi['group'] = tumor_classification_df[gene].apply(map_class_to_group)
    file_name = f"{tumor}_{gene}_survival_data.csv"
    df_for_lorsi.to_csv(path.join(tumor_data_dir, file_name), index=False)

def calc_orig_p_value_diff(row):
  return f"{round((((row['LoRSI_orig_pvalue'] - row['R_pvalue'])/row['R_pvalue'])*100), 2)}%"

In [ ]:
classification_files = list(pathlib.Path(results_dir).glob("*logrank_classification_results.txt"))
for file_ in classification_files:
  tumor = file_.name.split('_')[0]
  tumor_classification_df = pd.read_csv(file_, sep="\t")
  prepare_tumor_data_for_lorsi(tumor, tumor_classification_df)

In [ ]:
classification_files = list(pathlib.Path(results_dir).glob("*logrank_classification_results.txt"))
total_cases = 0
for file_ in classification_files:
  tumor = file_.name.split('_')[0]
  if tumor in site_list:
    tumor_classification_df = pd.read_csv(file_, sep="\t")
    num_cases = tumor_classification_df.shape[0]
    print(f"{tumor} has {num_cases} cases")
    total_cases += num_cases
print(f"Total number of cases is {total_cases}")

Corpus uteri has 545 cases
Brain has 666 cases
Pancreas has 176 cases
Hematopoietic and reticuloendothelial systems has 306 cases
Kidney has 883 cases
Bronchus and lung has 1013 cases
Skin has 463 cases
Cervix uteri has 291 cases
Thyroid gland has 503 cases
Prostate gland has 493 cases
Liver and intrahepatic bile ducts has 398 cases
Colon has 440 cases
Ovary has 377 cases
Adrenal gland has 227 cases
Lymph nodes has 29 cases
Stomach has 365 cases
Breast has 1073 cases
Bladder has 405 cases
Total number of cases is 8653


# Multitest correction (FDR)

In [ ]:
from statsmodels.stats.multitest import fdrcorrection as fdr

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from posix import listdir
fdr_dir = path.join(work_dir, 'fdr_adjusted_results')
if not path.exists(fdr_dir):
  os.mkdir(fdr_dir)

for site in site_list:
  print(site)
  failed_genes = []
  passed_genes = []
  site_cutoffs_dir = path.join(results_dir, f"{site}_cutoffs")
  if not path.exists(site_cutoffs_dir):
    print(f"{site_cutoffs_dir} doesn't exists")
  else:
    cutoffs_files = pathlib.Path(site_cutoffs_dir).glob("*_cutoff_table.txt")
    for cutoffs_file in cutoffs_files:
      gene = cutoffs_file.name.split('_')[1]
      df = pd.read_csv(cutoffs_file, sep="\t", usecols=["pvalue"]) 
      np_pvalues = df[np.invert(np.isnan(df['pvalue']))]['pvalue'].to_numpy()
      np_pvalues.sort()
      reject, fdr_pvalues = fdr(np_pvalues)
      reject = list(reject)
      try:
        first_false = reject.index(False)
      except Exception as e:
        first_false = df.shape[0]
      if (first_false == 0) or (first_false/df.shape[0] > 1/10) :
        failed_genes.append(gene)
      else:
        passed_genes.append(gene)
    print(f"Failed {len(failed_genes)}: {failed_genes}")
    print(f"Passed {len(passed_genes)}: {passed_genes}")     



# df.drop(columns=['Unnamed: 0', 'V2', 'V3', 'V4', 'Breast_HR',
#        'Breast_CI_low', 'Breast_CI_high', 'Breast_cutoff'], inplace=True)
# 
# df['reject'], df['fdr_pvalue'] = fdr(df['Breast_pvalue'].values)
# df.to_csv(path.join(fdr_dir, 'Breast_fdr_pvalues.csv'))

Adrenal gland
Failed 631: ['PGLS', 'PTK2B', 'MMP2', 'RAF1', 'CYB561D1', 'MSH2', 'CBX3', 'PRC1', 'ITGB1', 'KDR', 'PRPS1', 'SPHK1', 'PAK1', 'DAPK1', 'ATP5I', 'PMS1', 'XAB2', 'HDAC1', 'PARP3', 'NDUFV2', 'RNF168', 'XRCC4', 'JUNB', 'EZR', 'RPA3', 'RASSF1', 'CDKN2D', 'BMP2', 'ATRIP', 'RELA', 'KAT5', 'CS', 'ANG', 'CXCL12', 'NDUFB6', 'CDK7', 'WRAP53', 'ATM', 'CXCL5', 'NDUFB4', 'GYS2', 'XRCC1', 'NDUFB5', 'MMP1', 'PDHB', 'XRCC6', 'POLD3', 'TOPBP1', 'XPA', 'EWSR1', 'HK3', 'RAD54L', 'PRKCA', 'CTTN', 'PARP1', 'CXCL13', 'UQCRC1', 'NDUFA10', 'SMAD4', 'MYL9', 'ETS2', 'PEA15', 'RB1', 'CD82', 'RNF8', 'PHKG1', 'TERF2', 'PDK4', 'HDAC7', 'CDKN2B', 'CYC1', 'DAXX', 'MSH6', 'PNKP', 'CSNK2A2', 'MAPK1', 'TWIST1', 'CHEK2', 'PTN', 'MYB', 'RPIA', 'COX8A', 'CRADD', 'H6PD', 'COX7A2L', 'VEGFD', 'DNMT1', 'MGMT', 'PRPS2', 'BBC3', 'AMOT', 'ZEB1', 'CTSK', 'CDC20', 'RET', 'MLH3', 'MDH1B', 'FGF1', 'DDB1', 'PRKCB', 'RBBP8', 'PGM2', 'NDUFS4', 'PGM3', 'ERCC4', 'CCND1', 'JUND', 'VCL', 'DLD', 'SMUG1', 'JUN', 'ACTR2', 'CASP9', '

In [ ]:
cox_os = '/content/drive/MyDrive/Msc/Final project/Pancancer survival analysis of cancer hallmark genes/results/OS_survival_results.txt'
logrank_os = '/content/drive/MyDrive/Msc/Final project/Pancancer survival analysis of cancer hallmark genes/results/logrank_OS_survival_results.csv'

cox_os_df = pd.read_csv(cox_os, sep="\t", index_col="V1")
logrank_os_df = pd.read_csv(logrank_os, sep="\t", index_col="V1")
cox_os_df = cox_os_df.loc[:, 'Breast_cutoff']
logrank_os_df = logrank_os_df.loc[:, 'Breast_cutoff']
merged_df = pd.DataFrame(columns=['COX_cutoff', 'logrank_cutoff'], index=cox_os_df.index)
merged_df = merged_df.merge(cox_os_df, right_index=True, left_index=True)
merged_df.rename(columns={'Breast_cutoff':'cox_Breast_cutoff'}, inplace=True)
merged_df = merged_df.merge(logrank_os_df, right_index=True, left_index=True)
merged_df.rename(columns={'Breast_cutoff':'logrank_Breast_cutoff'}, inplace=True)
merged_df.head()

In [ ]:
my_site_list = sorted(os.listdir(data_for_lorsi_dir))[-6:]
my_site_list

['Ovary', 'Pancreas', 'Prostate gland', 'Skin', 'Stomach', 'Thyroid gland']

In [ ]:
my_site_list = sorted(os.listdir(data_for_lorsi_dir))[-6:]
my_site_dict = {}
for site in my_site_list:
  try:
    my_site_dict[site] = cancer_dict[site]
  except Exception as e:
    print(e)
# my_site_list = ['Pancreas', 'Stomach']
report_name = 'report_amir.csv'
Pancancer_report_producer.run(my_site_dict, report_name, work_dir)

Extracting Cancer type: Skin:   0%|          | 0/636 [00:00<?, ?it/s]

Extracting Cancer type: Pancreas:   0%|          | 0/649 [00:00<?, ?it/s]

Extracting Cancer type: Thyroid gland:   0%|          | 0/635 [00:00<?, ?it/s]

Extracting Cancer type: Stomach:   0%|          | 0/650 [00:00<?, ?it/s]

Extracting Cancer type: Ovary:   0%|          | 0/648 [00:00<?, ?it/s]

Extracting Cancer type: Prostate gland:   0%|          | 0/642 [00:00<?, ?it/s]